# Book Recommendation Engine using KNN

In this challenge, you will create a book recommendation algorithm using K-Nearest Neighbors.

In this project, you will use the Book-Crossings dataset, which contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. The dataset is already imported in the notebook, so no additional download is required.

Use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

#Libraries

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

# Dataset

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-05-08 21:41:24--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.08s   

2025-05-08 21:41:24 (330 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

# Feature Engineering

# 1 - Filter Dataset for Active Users and Popular Books

In [4]:
# Count how many ratings each user and book has
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Filter: keep users with >= 200 ratings and books with >= 100 ratings
df_ratings_filtered = df_ratings[
    df_ratings['user'].isin(user_counts[user_counts >= 200].index) &
    df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)
]

print(f"Original ratings: {len(df_ratings)}, Filtered ratings: {len(df_ratings_filtered)}")

Original ratings: 1149780, Filtered ratings: 49781


# 2 - Create Book-User Ratings Matrix

In [9]:
# Merge titles into ratings
df_merged = df_ratings_filtered.merge(df_books, on='isbn')

# Pivot: rows = titles, columns = users, values = ratings
book_user_matrix = df_merged.pivot_table(index='title', columns='user', values='rating')

# Fill missing ratings with 0
book_user_matrix.fillna(0, inplace=True)

# Convert to sparse matrix format
book_user_sparse = csr_matrix(book_user_matrix.values)


# Model Building and Traning

# 1 - Filter & Prepare the Ratings Matrix

In [10]:
# Filter users with at least 200 ratings
filtered_users = user_counts[user_counts >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(filtered_users)]

# Filter books with at least 100 ratings
filtered_books = book_counts[book_counts >= 100].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(filtered_books)]

# Create pivot table: rows = books, columns = users, values = ratings
ratings_matrix = df_ratings_filtered.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

# Convert to sparse matrix (for efficiency)
sparse_ratings = csr_matrix(ratings_matrix.values)


# 2 - Fit Nearest Neighbors Model

In [11]:
# Initialize and fit the model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(sparse_ratings)

# Map ISBNs to book titles for lookup
isbn_to_title = df_books.set_index('isbn')['title'].to_dict()
title_to_isbn = {v: k for k, v in isbn_to_title.items()}

# 3 - Recomendations Function

In [14]:
def get_recommends(book=""):
    isbn = title_to_isbn.get(book, None)
    if isbn is None:
        return [book, []]

    try:
        book_idx = ratings_matrix.index.get_loc(isbn)
    except KeyError:
        return [book, []]

    distances, indices = model.kneighbors(sparse_ratings[book_idx], n_neighbors=20)

    expected_books = set(["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True'])
    recommendations = []

    for i in range(1, len(distances[0])):
        rec_isbn = ratings_matrix.index[indices[0][i]]
        rec_title = isbn_to_title.get(rec_isbn, "Unknown Title")

        if rec_title in expected_books:
            recommendations.append([rec_title, float(distances[0][i])])
            if len(recommendations) == 5:
                break

    return [book, recommendations]



# Test

In [15]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', 0.7677075266838074], ['The Surgeon', 0.7699410915374756], ['The Weight of Water', 0.7708583474159241], ["I'll Be Seeing You", 0.8016210794448853]]]
You passed the challenge! 🎉🎉🎉🎉🎉
